In [1]:
%matplotlib inline

import torch
import torchvision
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict

from data_utils import CustomImageDataset, split_image_data
from data_utils import get_default_data_transforms
from models import ConvNet
from fl_devices import Server, Client
from helper import ExperimentLogger, display_train_stats

from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA


torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
# helper functions

# feature_matrix:
# each row is flatten dWs from a client
# helper functions

# detect_adv_idx: adverary indices detected by server
# gt_adv_idx: ground-truth indices
def check_detect(detect_adv_idx, gt_adv_idx):
    intersection = [idx for idx in gt_adv_idx if idx in detect_adv_idx]
    if len(intersection) > 0:
        return True
    else:
        return False
    
# feature_matrix:
# each row is flatten dWs from a client
def generate_feature_matrix(dW_dicts):
    with torch.no_grad():
        rows = []
        
        for dW_dict in dW_dicts:
            row = torch.empty(0).to(device)
            for key, value in dW_dict.items():
                row = torch.cat((row, value.flatten()), 0)
            rows.append(row)
            
        matrix = torch.stack(rows, 0)
        if device is "cpu":
            return matrix.numpy()
        else:
            return matrix.cpu().numpy()
        
def print_labels(labels):
    string = []
    for idx, label in enumerate(labels):
        string.append(str(idx)+': '+str(label))
    print('\t'.join(string))
    
def print_outliers(labels):
    outlier_idx = np.argwhere(labels == -1).flatten()
    print(outlier_idx)
    
def print_distance(feature_matrix, metric):
    distance = pairwise_distances(feature_matrix,metric=metirc)
    return distance 

def handle_adversary(adv_idx, handle, weights, reg_factor):
    if handle == None:
        return weights
    elif handle == 'remove':
        weights[adv_idx] = 0
        return weights 
    elif handle == 'reg':
        weights[adv_idx] = weights[adv_idx] * reg_factor
        return weights
    return weights

In [65]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 3
N_ADV_OPP = 0
N_ADV_SWAP = 0

# hyperparemeters
TOTAL_TRIAL = 1
TOTAL_ROUND = 30
DETECT_ROUND = 10

ADV_HANDLE = [None, 'remove', 'reg']

esp = 0.8
min_samples =2
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)
for handle in ADV_HANDLE:
  for trial in range(TOTAL_TRIAL):
    data = datasets.MNIST(root='./',download=True)
    train_frac = 0.2
    test_frac = 0.2 
    train_num = int(train_frac * len(data))
    test_num = int(test_frac * len(data))
    idcs = np.random.permutation(len(data))
    train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
    train_labels = data.train_labels.numpy()
    clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
    train_trans, val_trans = get_default_data_transforms("EMNIST")
    client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

    test_data = data.test_data[train_num:train_num+test_num]
    test_labels = train_labels[train_num:train_num+test_num]
    test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

    # Assign client modes
    clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
              for i, dat in enumerate(client_data)]
    client_indx = np.random.permutation(len(clients))
    offset = 0
    adv_random = client_indx[0:N_ADV_RANDOM]
    offset += N_ADV_RANDOM
    adv_opp = client_indx[offset:offset + N_ADV_OPP]
    offset += N_ADV_OPP
    adv_swap = client_indx[offset:offset+N_ADV_SWAP]
    offset += N_ADV_SWAP
    adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
    for i in adv_random:
      clients[i].client_mode = 'random'

    for i in adv_opp:
      clients[i].client_mode = 'opposite'

    for i in adv_swap:
      clients[i].client_mode = 'swap'

    # print out each client and its mode
    for idx, client in enumerate(clients):
      print('{}: {}'.format(idx, client.client_mode))

    server = Server(ConvNet, test_data)
    weights = np.ones(len(clients))
    for round in range(TOTAL_ROUND):
      if round == 0:
        for client in clients:
          client.synchronize_with_server(server)
      
      
      participating_clients = server.select_clients(clients, frac=1.0)

      for client in participating_clients:
          train_stats = client.compute_weight_update(epochs=1)
          client.reset()
        
      if round + 1 == DETECT_ROUND:
          # generate feature matrix for clustering
          client_dW_dicts = [client.dW for client in clients]
          feature_matrix = generate_feature_matrix(client_dW_dicts)

          # detect adversary using clustering
          clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
          adv_idx_label = np.argwhere(clustering_label == -1).flatten()

          # update clients by handling adversary detected
          weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

      # aggregate weight updates; copy new weights to clients
      server.aggregate_wieght_updates_weight(clients, weights)
      server.copy_weights(clients)
                
    # evaluate model performance after all the rounds
    acc_clients = [client.evaluate() for client in clients]

    acc_s = []
    for i, acc in enumerate(acc_clients):
      if i not in adv_idx:
        acc_s.append(acc)
  
    model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
    print("handle %s, trial %d")
    print(acc_s)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Data split:
 - Client 0: [96 96 96 96  0  0  0  0  0 96]
 - Client 1: [ 0  0  0  0 96 96 96 96 96  0]
 - Client 2: [96 96 96  0  0  0  0  0 96 96]
 - Client 3: [96 96  0  0  0  0  0 96 96 96]
 - Client 4: [96 96 96 96  0  0  0  0  0 96]
 - Client 5: [ 0 96 96 96 96 96  0  0  0  0]
 - Client 6: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 7: [96 96 96  0  0  0  0  0 96 96]
 - Client 8: [96 96  0  0  0  0  0 96 96 96]
 - Client 9: [96  0  0  0  0  0 96 96 96 96]
 - Client 10: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 11: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 12: [ 0 96 96 96 96 96  0  0  0  0]
 - Client 13: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 14: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 15: [96 96 96  0  0  0  0  0 96 96]
 - Client 16: [ 0  0  0  0 96 96 96 96 96  0]
 - Client 17: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 18: [96  0  0  0  0  0 96 96 96 96]
 - Client 19: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 20: [42  0  0  0  0 71 96 79 96 96]
 - Client 21: [ 0 96 96 96 96  0

KeyboardInterrupt: ignored

**Experiment 0**: Baseline without attack

In [66]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 0
N_ADV_OPP = 0
N_ADV_SWAP = 0

# hyperparemeters
TOTAL_TRIAL = 5
TOTAL_ROUND = 30
DETECT_ROUND = 10

# ADV_HANDLE = [None, 'remove', 'reg']
handle = None

esp = 0.8
min_samples =2
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

for trial in range(TOTAL_TRIAL):
  data = datasets.MNIST(root='./',download=True)
  train_frac = 0.2
  test_frac = 0.2 
  train_num = int(train_frac * len(data))
  test_num = int(test_frac * len(data))
  idcs = np.random.permutation(len(data))
  train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
  train_labels = data.train_labels.numpy()
  clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
  train_trans, val_trans = get_default_data_transforms("EMNIST")
  client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

  test_data = data.test_data[train_num:train_num+test_num]
  test_labels = train_labels[train_num:train_num+test_num]
  test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

  # Assign client modes
  clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
            for i, dat in enumerate(client_data)]
  client_indx = np.random.permutation(len(clients))
  offset = 0
  adv_random = client_indx[0:N_ADV_RANDOM]
  offset += N_ADV_RANDOM
  adv_opp = client_indx[offset:offset + N_ADV_OPP]
  offset += N_ADV_OPP
  adv_swap = client_indx[offset:offset+N_ADV_SWAP]
  offset += N_ADV_SWAP
  adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
  for i in adv_random:
    clients[i].client_mode = 'random'

  for i in adv_opp:
    clients[i].client_mode = 'opposite'

  for i in adv_swap:
    clients[i].client_mode = 'swap'

  # print out each client and its mode
  for idx, client in enumerate(clients):
    print('{}: {}'.format(idx, client.client_mode))

  server = Server(ConvNet, test_data)
  weights = np.ones(len(clients))
  for round in range(TOTAL_ROUND):
    if round == 0:
      for client in clients:
        client.synchronize_with_server(server)
    
    
    participating_clients = server.select_clients(clients, frac=1.0)

    for client in participating_clients:
        train_stats = client.compute_weight_update(epochs=1)
        client.reset()
      
    if round + 1 == DETECT_ROUND:
        # generate feature matrix for clustering
        client_dW_dicts = [client.dW for client in clients]
        feature_matrix = generate_feature_matrix(client_dW_dicts)

        # detect adversary using clustering
        clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
        adv_idx_label = np.argwhere(clustering_label == -1).flatten()

        # update clients by handling adversary detected
        weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

    # aggregate weight updates; copy new weights to clients
    server.aggregate_wieght_updates_weight(clients, weights)
    server.copy_weights(clients)
              
  # evaluate model performance after all the rounds
  acc_clients = [client.evaluate() for client in clients]

  acc_s = []
  for i, acc in enumerate(acc_clients):
    if i not in adv_idx:
      acc_s.append(acc)

  model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
  print("handle %s, trial %d")
  print(acc_s)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Data split:
 - Client 0: [96  0  0  0  0  0 96 96 96 96]
 - Client 1: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 2: [ 0  0  0  0 96 96 96 96 96  0]
 - Client 3: [96 96 96  0  0  0  0  0 96 96]
 - Client 4: [96 96 96 96 96  0  0  0  0  0]
 - Client 5: [96 96 96 96  0  0  0  0  0 96]
 - Client 6: [96 96 96  0  0  0  0  0 96 96]
 - Client 7: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 8: [96 96  0  0  0  0  0 96 96 96]
 - Client 9: [96 96 96 96 96  0  0  0  0  0]
 - Client 10: [96 96 96 96 96  0  0  0  0  0]
 - Client 11: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 12: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 13: [96  0  0  0  0  0 96 96 96 96]
 - Client 14: [96  0  0  0  0  0 96 96 96 96]
 - Client 15: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 16: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 17: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 18: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 19: [ 0  0 84 96 80 96 71 53  0  0]
 - Client 20: [96 96  0  0  0  0  0 96 96 96]
 - Client 21: [96 96  0 96  0 96

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


0: normal
1: normal
2: normal
3: normal
4: normal
5: normal
6: normal
7: normal
8: normal
9: normal
10: normal
11: normal
12: normal
13: normal
14: normal
15: normal
16: normal
17: normal
18: normal
19: normal
20: normal
21: normal
22: normal
23: normal
24: normal
handle %s, trial %d
[0.7291666666666666, 0.7604166666666666, 0.8958333333333334, 0.8020833333333334, 0.8020833333333334, 0.8229166666666666, 0.7708333333333334, 0.84375, 0.8229166666666666, 0.8333333333333334, 0.6979166666666666, 0.8020833333333334, 0.7708333333333334, 0.7395833333333334, 0.8020833333333334, 0.78125, 0.7291666666666666, 0.8229166666666666, 0.8125, 0.78125, 0.8229166666666666, 0.8645833333333334, 0.8020833333333334, 0.8541666666666666, 0.9166666666666666]
Data split:
 - Client 0: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 1: [96 96 96  0  0  0  0  0 96 96]
 - Client 2: [96 96 96 96 96  0  0  0  0  0]
 - Client 3: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 4: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 5: [96  0  0

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle %s, trial %d
[0.7916666666666666, 0.8229166666666666, 0.8541666666666666, 0.875, 0.7604166666666666, 0.8125, 0.8125, 0.8333333333333334, 0.8229166666666666, 0.7291666666666666, 0.78125, 0.8541666666666666, 0.84375, 0.8541666666666666, 0.78125, 0.8854166666666666, 0.78125, 0.8958333333333334, 0.8020833333333334, 0.8645833333333334, 0.8333333333333334, 0.7916666666666666, 0.78125, 0.8541666666666666, 0.9270833333333334]
Data split:
 - Client 0: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 1: [ 0  0 96 96 96 96 96  0  0  0]
 - Client 2: [96 96 96 96  0  0  0  0  0 96]
 - Client 3: [96 96 96 96 96  0  0  0  0  0]
 - Client 4: [ 0 96 96 96 96 96  0  0  0  0]
 - Client 5: [96 96  0  0  0  0  0 96 96 96]
 - Client 6: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 7: [96 96 96  0  0  0  0  0 96 96]
 - Client 8: [96 96 96  0  0  0  0  0 96 96]
 - Client 9: [96 96 96 96 96  0  0  0  0  0]
 - Client 10: [96 96 96  0  0  0  0  0 96 96]
 - Client 11: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 12: [ 0

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle %s, trial %d
[0.78125, 0.8645833333333334, 0.8125, 0.875, 0.875, 0.84375, 0.8854166666666666, 0.8645833333333334, 0.84375, 0.8541666666666666, 0.8854166666666666, 0.8333333333333334, 0.8020833333333334, 0.8229166666666666, 0.84375, 0.8958333333333334, 0.8020833333333334, 0.8229166666666666, 0.8125, 0.7604166666666666, 0.8020833333333334, 0.8958333333333334, 0.8541666666666666, 0.78125, 0.8333333333333334]
Data split:
 - Client 0: [96 96 96 96 96  0  0  0  0  0]
 - Client 1: [96 96 96  0  0  0  0  0 96 96]
 - Client 2: [96 96 96 96  0  0  0  0  0 96]
 - Client 3: [ 0  0  0 96 96 96 96 96  0  0]
 - Client 4: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 5: [ 0  0  0  0  0 96 96 96 96 96]
 - Client 6: [96 96 96  0  0  0  0  0 96 96]
 - Client 7: [96 96  0  0  0  0  0 96 96 96]
 - Client 8: [96  0  0  0  0  0 96 96 96 96]
 - Client 9: [96 96 96 96 96  0  0  0  0  0]
 - Client 10: [96 96 96  0  0  0  0  0 96 96]
 - Client 11: [ 0  0  0  0 96 96 96 96 96  0]
 - Client 12: [ 0  0  0 96 96 

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle %s, trial %d
[0.75, 0.8229166666666666, 0.8125, 0.78125, 0.8020833333333334, 0.8333333333333334, 0.8229166666666666, 0.8333333333333334, 0.875, 0.8645833333333334, 0.8854166666666666, 0.8333333333333334, 0.7916666666666666, 0.84375, 0.875, 0.78125, 0.8229166666666666, 0.7708333333333334, 0.8229166666666666, 0.8333333333333334, 0.8229166666666666, 0.7291666666666666, 0.78125, 0.90625, 0.9479166666666666]


**Experiment 1.1**: Random Attack, None detect

In [61]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 3
N_ADV_OPP = 0
N_ADV_SWAP = 0

# hyperparemeters
TOTAL_TRIAL = 5
TOTAL_ROUND = 30
DETECT_ROUND = 10

# ADV_HANDLE = [None, 'remove', 'reg']
handle = None

esp = 0.8
min_samples =2
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

for trial in range(TOTAL_TRIAL):
  data = datasets.MNIST(root='./',download=True)
  train_frac = 0.5
  test_frac = 0.2 
  train_num = int(train_frac * len(data))
  test_num = int(test_frac * len(data))
  idcs = np.random.permutation(len(data))
  train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
  train_labels = data.train_labels.numpy()
  clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
  train_trans, val_trans = get_default_data_transforms("EMNIST")
  client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

  test_data = data.test_data[train_num:train_num+test_num]
  test_labels = train_labels[train_num:train_num+test_num]
  test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

  # Assign client modes
  clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
            for i, dat in enumerate(client_data)]
  client_indx = np.random.permutation(len(clients))
  offset = 0
  adv_random = client_indx[0:N_ADV_RANDOM]
  offset += N_ADV_RANDOM
  adv_opp = client_indx[offset:offset + N_ADV_OPP]
  offset += N_ADV_OPP
  adv_swap = client_indx[offset:offset+N_ADV_SWAP]
  offset += N_ADV_SWAP
  adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
  for i in adv_random:
    clients[i].client_mode = 'random'

  for i in adv_opp:
    clients[i].client_mode = 'opposite'

  for i in adv_swap:
    clients[i].client_mode = 'swap'

  # print out each client and its mode
  for idx, client in enumerate(clients):
    print('{}: {}'.format(idx, client.client_mode))

  server = Server(ConvNet, test_data)
  weights = np.ones(len(clients))
  for round in range(TOTAL_ROUND):
    if round == 0:
      for client in clients:
        client.synchronize_with_server(server)
    
    
    participating_clients = server.select_clients(clients, frac=1.0)

    for client in participating_clients:
        train_stats = client.compute_weight_update(epochs=1)
        client.reset()
      
    if round + 1 == DETECT_ROUND:
        # generate feature matrix for clustering
        client_dW_dicts = [client.dW for client in clients]
        feature_matrix = generate_feature_matrix(client_dW_dicts)

        # detect adversary using clustering
        clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
        adv_idx_label = np.argwhere(clustering_label == -1).flatten()

        # update clients by handling adversary detected
        weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

    # aggregate weight updates; copy new weights to clients
    server.aggregate_wieght_updates_weight(clients, weights)
    server.copy_weights(clients)
              
  # evaluate model performance after all the rounds
  acc_clients = [client.evaluate() for client in clients]

  acc_s = []
  for i, acc in enumerate(acc_clients):
    if i not in adv_idx:
      acc_s.append(acc)

  model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
  print("handle %s, trial %d"%(handle, trial))
  print(acc_s)
print("average performance %f"%(np.mean(model_performance[handle])))

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Data split:
 - Client 0: [  0   0   0   0   0 240 240 240 240 240]
 - Client 1: [  0   0 240 240 240 240 240   0   0   0]
 - Client 2: [240   0   0   0   0   0 240 240 240 240]
 - Client 3: [240 240 240 240 240   0   0   0   0   0]
 - Client 4: [  0   0   0   0 240 240 240 240 240   0]
 - Client 5: [  0   0 240 240 240 240 240   0   0   0]
 - Client 6: [240 240 240   0   0   0   0   0 240 240]
 - Client 7: [240 240 240 240   0   0   0   0   0 240]
 - Client 8: [240 240 240   0   0   0   0   0 240 240]
 - Client 9: [240 240 240 240 240   0   0   0   0   0]
 - Client 10: [240 240 240 240 240   0   0   0   0   0]
 - Client 11: [  0   0   0   0 240 240 240 240 240   0]
 - Client 12: [240 240 240 240 240   0   0   0   0   0]
 - Client 13: [  0   0   0   0   0 240 240 240 240 240]
 - Client 14: [240 240 240 240 240   0   0   0   0   0]
 - Client 15: [240 240   0   0   0   0   0 240 240 240]
 - Client 16: [240 240 240 240   0   0   0   0   0 240]
 - Client 17: [  0 240 240 240 240 240   0   0

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


0: normal
1: normal
2: normal
3: normal
4: normal
5: normal
6: normal
7: random
8: normal
9: normal
10: normal
11: random
12: normal
13: normal
14: normal
15: normal
16: normal
17: normal
18: normal
19: normal
20: normal
21: normal
22: normal
23: random
24: normal
handle None, trial 1
[0.8291666666666667, 0.8458333333333333, 0.8083333333333333, 0.8291666666666667, 0.8458333333333333, 0.7708333333333334, 0.775, 0.7541666666666667, 0.7416666666666667, 0.8375, 0.775, 0.7625, 0.7625, 0.8333333333333334, 0.8708333333333333, 0.7958333333333333, 0.775, 0.7833333333333333, 0.7125, 0.8416666666666667, 0.8125, 0.7916666666666666]
Data split:
 - Client 0: [  0   0   0   0 240 240 240 240 240   0]
 - Client 1: [  0   0   0   0   0 240 240 240 240 240]
 - Client 2: [240 240   0   0   0   0   0 240 240 240]
 - Client 3: [  0   0   0   0 240 240 240 240 240   0]
 - Client 4: [  0   0   0   0 240 240 240 240 240   0]
 - Client 5: [  0 240 240 240 240 240   0   0   0   0]
 - Client 6: [240   0   0   0 

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 2
[0.6291666666666667, 0.7708333333333334, 0.8583333333333333, 0.7041666666666667, 0.6333333333333333, 0.8416666666666667, 0.7333333333333333, 0.6791666666666667, 0.8166666666666667, 0.8333333333333334, 0.8166666666666667, 0.65, 0.6666666666666666, 0.8041666666666667, 0.6375, 0.8416666666666667, 0.7541666666666667, 0.6875, 0.775, 0.825, 0.8208333333333333, 0.9041666666666667]
Data split:
 - Client 0: [240 240 240 240 240   0   0   0   0   0]
 - Client 1: [  0   0   0   0   0 240 240 240 240 240]
 - Client 2: [  0   0   0   0 240 240 240 240 240   0]
 - Client 3: [  0 240 240 240 240 240   0   0   0   0]
 - Client 4: [240 240   0   0   0   0   0 240 240 240]
 - Client 5: [240 240 240   0   0   0   0   0 240 240]
 - Client 6: [  0   0   0   0 240 240 240 240 240   0]
 - Client 7: [240   0   0   0   0   0 240 240 240 240]
 - Client 8: [240   0   0   0   0   0 240 240 240 240]
 - Client 9: [  0 240 240 240 240 240   0   0   0   0]
 - Client 10: [  0   0   0   0   0 240 2

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 3
[0.7583333333333333, 0.8041666666666667, 0.775, 0.6583333333333333, 0.8166666666666667, 0.8375, 0.7375, 0.85, 0.8458333333333333, 0.6791666666666667, 0.7208333333333333, 0.6375, 0.6708333333333333, 0.7125, 0.8583333333333333, 0.6791666666666667, 0.8583333333333333, 0.6958333333333333, 0.8291666666666667, 0.6666666666666666, 0.8708333333333333, 0.7125]
Data split:
 - Client 0: [  0   0   0   0 240 240 240 240 240   0]
 - Client 1: [  0   0   0 240 240 240 240 240   0   0]
 - Client 2: [  0 240 240 240 240 240   0   0   0   0]
 - Client 3: [240 240 240 240 240   0   0   0   0   0]
 - Client 4: [  0 240 240 240 240 240   0   0   0   0]
 - Client 5: [240 240 240 240   0   0   0   0   0 240]
 - Client 6: [  0   0   0   0 240 240 240 240 240   0]
 - Client 7: [240   0   0   0   0   0 240 240 240 240]
 - Client 8: [  0   0   0 240 240 240 240 240   0   0]
 - Client 9: [  0 240 240 240 240 240   0   0   0   0]
 - Client 10: [  0   0 240 240 240 240 240   0   0   0]
 - Clie

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 4
[0.7625, 0.7291666666666666, 0.7791666666666667, 0.8125, 0.7708333333333334, 0.7916666666666666, 0.7333333333333333, 0.7833333333333333, 0.7125, 0.825, 0.7583333333333333, 0.6875, 0.7166666666666667, 0.8333333333333334, 0.8041666666666667, 0.7916666666666666, 0.8458333333333333, 0.7416666666666667, 0.7916666666666666, 0.8041666666666667, 0.7833333333333333, 0.8833333333333333]
average performance 0.704015


**Experiment 1.2**: Flip Attack, None detect

In [62]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 0
N_ADV_OPP = 0
N_ADV_SWAP = 3

# hyperparemeters
TOTAL_TRIAL = 5
TOTAL_ROUND = 30
DETECT_ROUND = 10

# ADV_HANDLE = [None, 'remove', 'reg']
handle = None

esp = 0.8
min_samples =2
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

for trial in range(TOTAL_TRIAL):
  data = datasets.MNIST(root='./',download=True)
  train_frac = 0.5
  test_frac = 0.2 
  train_num = int(train_frac * len(data))
  test_num = int(test_frac * len(data))
  idcs = np.random.permutation(len(data))
  train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
  train_labels = data.train_labels.numpy()
  clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
  train_trans, val_trans = get_default_data_transforms("EMNIST")
  client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

  test_data = data.test_data[train_num:train_num+test_num]
  test_labels = train_labels[train_num:train_num+test_num]
  test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

  # Assign client modes
  clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
            for i, dat in enumerate(client_data)]
  client_indx = np.random.permutation(len(clients))
  offset = 0
  adv_random = client_indx[0:N_ADV_RANDOM]
  offset += N_ADV_RANDOM
  adv_opp = client_indx[offset:offset + N_ADV_OPP]
  offset += N_ADV_OPP
  adv_swap = client_indx[offset:offset+N_ADV_SWAP]
  offset += N_ADV_SWAP
  adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
  for i in adv_random:
    clients[i].client_mode = 'random'

  for i in adv_opp:
    clients[i].client_mode = 'opposite'

  for i in adv_swap:
    clients[i].client_mode = 'swap'

  # print out each client and its mode
  for idx, client in enumerate(clients):
    print('{}: {}'.format(idx, client.client_mode))

  server = Server(ConvNet, test_data)
  weights = np.ones(len(clients))
  for round in range(TOTAL_ROUND):
    if round == 0:
      for client in clients:
        client.synchronize_with_server(server)
    
    
    participating_clients = server.select_clients(clients, frac=1.0)

    for client in participating_clients:
        train_stats = client.compute_weight_update(epochs=1)
        client.reset()
      
    if round + 1 == DETECT_ROUND:
        # generate feature matrix for clustering
        client_dW_dicts = [client.dW for client in clients]
        feature_matrix = generate_feature_matrix(client_dW_dicts)

        # detect adversary using clustering
        clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
        adv_idx_label = np.argwhere(clustering_label == -1).flatten()

        # update clients by handling adversary detected
        weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

    # aggregate weight updates; copy new weights to clients
    server.aggregate_wieght_updates_weight(clients, weights)
    server.copy_weights(clients)
              
  # evaluate model performance after all the rounds
  acc_clients = [client.evaluate() for client in clients]

  acc_s = []
  for i, acc in enumerate(acc_clients):
    if i not in adv_idx:
      acc_s.append(acc)

  model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
  print("handle %s, trial %d"%(handle, trial))
  print(acc_s)
print("average performance %f"%(np.mean(model_performance[handle])))

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Data split:
 - Client 0: [  0   0 240 240 240 240 240   0   0   0]
 - Client 1: [240 240   0   0   0   0   0 240 240 240]
 - Client 2: [240 240 240   0   0   0   0   0 240 240]
 - Client 3: [240 240 240   0   0   0   0   0 240 240]
 - Client 4: [  0   0   0 240 240 240 240 240   0   0]
 - Client 5: [240 240 240 240   0   0   0   0   0 240]
 - Client 6: [  0   0   0   0 240 240 240 240 240   0]
 - Client 7: [  0   0   0 240 240 240 240 240   0   0]
 - Client 8: [240 240 240   0   0   0   0   0 240 240]
 - Client 9: [  0   0 240 240 240 240 240   0   0   0]
 - Client 10: [240 240 240 240   0   0   0   0   0 240]
 - Client 11: [  0   0   0   0   0 240 240 240 240 240]
 - Client 12: [240 240   0   0   0   0   0 240 240 240]
 - Client 13: [240 240 240 240   0   0   0   0   0 240]
 - Client 14: [  0   0   0   0 240 240 240 240 240   0]
 - Client 15: [  0 240 240 240 240 240   0   0   0   0]
 - Client 16: [  0   0   0   0 240 240 240 240 240   0]
 - Client 17: [240 240 240 240 240   0   0   0

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 1
[0.8916666666666667, 0.9, 0.8208333333333333, 0.8416666666666667, 0.9041666666666667, 0.8208333333333333, 0.7958333333333333, 0.8333333333333334, 0.8458333333333333, 0.8791666666666667, 0.8208333333333333, 0.825, 0.85, 0.8666666666666667, 0.8, 0.8833333333333333, 0.8041666666666667, 0.8666666666666667, 0.8625, 0.8833333333333333, 0.8416666666666667, 0.8166666666666667]
Data split:
 - Client 0: [240 240 240   0   0   0   0   0 240 240]
 - Client 1: [240 240 240 240   0   0   0   0   0 240]
 - Client 2: [240   0   0   0   0   0 240 240 240 240]
 - Client 3: [  0   0 240 240 240 240 240   0   0   0]
 - Client 4: [240 240 240 240   0   0   0   0   0 240]
 - Client 5: [240 240 240 240   0   0   0   0   0 240]
 - Client 6: [  0 240 240 240 240 240   0   0   0   0]
 - Client 7: [240 240 240 240   0   0   0   0   0 240]
 - Client 8: [240 240 240 240   0   0   0   0   0 240]
 - Client 9: [  0 240 240 240 240 240   0   0   0   0]
 - Client 10: [240 240   0   0   0   0   0 24

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 2
[0.8583333333333333, 0.8458333333333333, 0.9125, 0.8416666666666667, 0.9, 0.8666666666666667, 0.825, 0.8458333333333333, 0.8208333333333333, 0.8833333333333333, 0.8541666666666666, 0.8583333333333333, 0.8291666666666667, 0.9, 0.8541666666666666, 0.875, 0.8791666666666667, 0.8958333333333334, 0.8875, 0.8958333333333334, 0.8458333333333333, 0.8458333333333333]
Data split:
 - Client 0: [240 240 240   0   0   0   0   0 240 240]
 - Client 1: [  0   0   0   0 240 240 240 240 240   0]
 - Client 2: [  0   0   0   0 240 240 240 240 240   0]
 - Client 3: [240 240   0   0   0   0   0 240 240 240]
 - Client 4: [  0   0   0   0 240 240 240 240 240   0]
 - Client 5: [240   0   0   0   0   0 240 240 240 240]
 - Client 6: [  0   0   0   0 240 240 240 240 240   0]
 - Client 7: [  0 240 240 240 240 240   0   0   0   0]
 - Client 8: [240 240 240 240   0   0   0   0   0 240]
 - Client 9: [240   0   0   0   0   0 240 240 240 240]
 - Client 10: [  0 240 240 240 240 240   0   0   0   0]


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 3
[0.6083333333333333, 0.48333333333333334, 0.4791666666666667, 0.6666666666666666, 0.4791666666666667, 0.4583333333333333, 0.4791666666666667, 0.5625, 0.5666666666666667, 0.4666666666666667, 0.6208333333333333, 0.49583333333333335, 0.4041666666666667, 0.6291666666666667, 0.6, 0.5083333333333333, 0.6458333333333334, 0.5875, 0.49583333333333335, 0.3625, 0.49166666666666664, 0.49583333333333335]
Data split:
 - Client 0: [  0   0   0   0   0 240 240 240 240 240]
 - Client 1: [  0   0   0 240 240 240 240 240   0   0]
 - Client 2: [240 240 240   0   0   0   0   0 240 240]
 - Client 3: [  0   0 240 240 240 240 240   0   0   0]
 - Client 4: [  0   0   0   0   0 240 240 240 240 240]
 - Client 5: [  0   0 240 240 240 240 240   0   0   0]
 - Client 6: [  0   0   0   0 240 240 240 240 240   0]
 - Client 7: [  0   0   0   0 240 240 240 240 240   0]
 - Client 8: [240   0   0   0   0   0 240 240 240 240]
 - Client 9: [240   0   0   0   0   0 240 240 240 240]
 - Client 10: [240 240

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 4
[0.8833333333333333, 0.8291666666666667, 0.9125, 0.8666666666666667, 0.8708333333333333, 0.8791666666666667, 0.9, 0.9125, 0.9208333333333333, 0.9166666666666666, 0.9, 0.8791666666666667, 0.8875, 0.9041666666666667, 0.9, 0.8625, 0.95, 0.8916666666666667, 0.9041666666666667, 0.8916666666666667, 0.875, 0.9125]
average performance 0.792424


**Experiment 1.3**: Opposite Attack, None detect

In [64]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 0
N_ADV_OPP = 3
N_ADV_SWAP = 0

# hyperparemeters
TOTAL_TRIAL = 5
TOTAL_ROUND = 30
DETECT_ROUND = 10

# ADV_HANDLE = [None, 'remove', 'reg']
handle = None

esp = 0.8
min_samples =2
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

for trial in range(TOTAL_TRIAL):
  data = datasets.MNIST(root='./',download=True)
  train_frac = 0.5
  test_frac = 0.2 
  train_num = int(train_frac * len(data))
  test_num = int(test_frac * len(data))
  idcs = np.random.permutation(len(data))
  train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
  train_labels = data.train_labels.numpy()
  clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
  train_trans, val_trans = get_default_data_transforms("EMNIST")
  client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

  test_data = data.test_data[train_num:train_num+test_num]
  test_labels = train_labels[train_num:train_num+test_num]
  test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

  # Assign client modes
  clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
            for i, dat in enumerate(client_data)]
  client_indx = np.random.permutation(len(clients))
  offset = 0
  adv_random = client_indx[0:N_ADV_RANDOM]
  offset += N_ADV_RANDOM
  adv_opp = client_indx[offset:offset + N_ADV_OPP]
  offset += N_ADV_OPP
  adv_swap = client_indx[offset:offset+N_ADV_SWAP]
  offset += N_ADV_SWAP
  adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
  for i in adv_random:
    clients[i].client_mode = 'random'

  for i in adv_opp:
    clients[i].client_mode = 'opposite'

  for i in adv_swap:
    clients[i].client_mode = 'swap'

  # print out each client and its mode
  for idx, client in enumerate(clients):
    print('{}: {}'.format(idx, client.client_mode))

  server = Server(ConvNet, test_data)
  weights = np.ones(len(clients))
  for round in range(TOTAL_ROUND):
    if round == 0:
      for client in clients:
        client.synchronize_with_server(server)
    
    
    participating_clients = server.select_clients(clients, frac=1.0)

    for client in participating_clients:
        train_stats = client.compute_weight_update(epochs=1)
        client.reset()
      
    if round + 1 == DETECT_ROUND:
        # generate feature matrix for clustering
        client_dW_dicts = [client.dW for client in clients]
        feature_matrix = generate_feature_matrix(client_dW_dicts)

        # detect adversary using clustering
        clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
        adv_idx_label = np.argwhere(clustering_label == -1).flatten()

        # update clients by handling adversary detected
        weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

    # aggregate weight updates; copy new weights to clients
    server.aggregate_wieght_updates_weight(clients, weights)
    server.copy_weights(clients)
              
  # evaluate model performance after all the rounds
  acc_clients = [client.evaluate() for client in clients]

  acc_s = []
  for i, acc in enumerate(acc_clients):
    if i not in adv_idx:
      acc_s.append(acc)

  model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
  print("handle %s, trial %d"%(handle, trial))
  print(acc_s)
print("average performance %f"%(np.mean(model_performance[handle])))

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Data split:
 - Client 0: [240 240 240 240 240   0   0   0   0   0]
 - Client 1: [  0   0   0 240 240 240 240 240   0   0]
 - Client 2: [  0   0   0   0   0 240 240 240 240 240]
 - Client 3: [240 240 240 240 240   0   0   0   0   0]
 - Client 4: [240   0   0   0   0   0 240 240 240 240]
 - Client 5: [240 240 240   0   0   0   0   0 240 240]
 - Client 6: [240   0   0   0   0   0 240 240 240 240]
 - Client 7: [  0   0   0   0 240 240 240 240 240   0]
 - Client 8: [  0   0   0 240 240 240 240 240   0   0]
 - Client 9: [  0   0   0   0 240 240 240 240 240   0]
 - Client 10: [240 240 240   0   0   0   0   0 240 240]
 - Client 11: [  0   0   0 240 240 240 240 240   0   0]
 - Client 12: [240 240 240   0   0   0   0   0 240 240]
 - Client 13: [240 240 240 240   0   0   0   0   0 240]
 - Client 14: [240 240 240 240 240   0   0   0   0   0]
 - Client 15: [  0   0   0   0 240 240 240 240 240   0]
 - Client 16: [240   0   0   0   0   0 240 240 240 240]
 - Client 17: [240 240   0   0   0   0   0 240

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 1
[0.75, 0.8458333333333333, 0.65, 0.6625, 0.8083333333333333, 0.7791666666666667, 0.675, 0.8416666666666667, 0.6791666666666667, 0.8416666666666667, 0.75, 0.7375, 0.7916666666666666, 0.8208333333333333, 0.7, 0.775, 0.8541666666666666, 0.8541666666666666, 0.8333333333333334, 0.8666666666666667, 0.8708333333333333, 0.875]
Data split:
 - Client 0: [240   0   0   0   0   0 240 240 240 240]
 - Client 1: [240 240 240 240   0   0   0   0   0 240]
 - Client 2: [240 240 240 240   0   0   0   0   0 240]
 - Client 3: [  0 240 240 240 240 240   0   0   0   0]
 - Client 4: [240 240 240 240 240   0   0   0   0   0]
 - Client 5: [  0   0   0 240 240 240 240 240   0   0]
 - Client 6: [240 240 240 240 240   0   0   0   0   0]
 - Client 7: [  0 240 240 240 240 240   0   0   0   0]
 - Client 8: [240 240 240 240 240   0   0   0   0   0]
 - Client 9: [  0   0 240 240 240 240 240   0   0   0]
 - Client 10: [240   0   0   0   0   0 240 240 240 240]
 - Client 11: [  0   0   0 240 240 240 2

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 2
[0.8583333333333333, 0.875, 0.8833333333333333, 0.925, 0.9, 0.8875, 0.9166666666666666, 0.8375, 0.9291666666666667, 0.8708333333333333, 0.8166666666666667, 0.9083333333333333, 0.8541666666666666, 0.8958333333333334, 0.8666666666666667, 0.8416666666666667, 0.85, 0.875, 0.8958333333333334, 0.9125, 0.8583333333333333, 0.8625]
Data split:
 - Client 0: [  0   0 240 240 240 240 240   0   0   0]
 - Client 1: [240   0   0   0   0   0 240 240 240 240]
 - Client 2: [240 240 240 240 240   0   0   0   0   0]
 - Client 3: [240 240 240   0   0   0   0   0 240 240]
 - Client 4: [  0   0 240 240 240 240 240   0   0   0]
 - Client 5: [240 240   0   0   0   0   0 240 240 240]
 - Client 6: [  0 240 240 240 240 240   0   0   0   0]
 - Client 7: [  0   0   0 240 240 240 240 240   0   0]
 - Client 8: [240 240 240 240   0   0   0   0   0 240]
 - Client 9: [240 240 240   0   0   0   0   0 240 240]
 - Client 10: [240   0   0   0   0   0 240 240 240 240]
 - Client 11: [240 240 240   0   0  

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 3
[0.6541666666666667, 0.7958333333333333, 0.825, 0.8791666666666667, 0.6416666666666667, 0.6708333333333333, 0.8, 0.8375, 0.8208333333333333, 0.8458333333333333, 0.6875, 0.6958333333333333, 0.6583333333333333, 0.6916666666666667, 0.675, 0.8208333333333333, 0.7208333333333333, 0.7291666666666666, 0.7375, 0.8291666666666667, 0.8458333333333333, 0.7666666666666667]
Data split:
 - Client 0: [  0   0   0   0   0 240 240 240 240 240]
 - Client 1: [240 240 240 240   0   0   0   0   0 240]
 - Client 2: [240 240 240   0   0   0   0   0 240 240]
 - Client 3: [  0   0   0 240 240 240 240 240   0   0]
 - Client 4: [  0   0 240 240 240 240 240   0   0   0]
 - Client 5: [  0   0   0 240 240 240 240 240   0   0]
 - Client 6: [  0   0   0   0   0 240 240 240 240 240]
 - Client 7: [240 240 240 240   0   0   0   0   0 240]
 - Client 8: [240 240   0   0   0   0   0 240 240 240]
 - Client 9: [240 240 240 240   0   0   0   0   0 240]
 - Client 10: [240 240 240 240   0   0   0   0   0 24

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


handle None, trial 4
[0.8458333333333333, 0.9041666666666667, 0.9541666666666667, 0.8333333333333334, 0.8083333333333333, 0.8583333333333333, 0.875, 0.9541666666666667, 0.9291666666666667, 0.8666666666666667, 0.7708333333333334, 0.7583333333333333, 0.7958333333333333, 0.8958333333333334, 0.875, 0.925, 0.8416666666666667, 0.8833333333333333, 0.8916666666666667, 0.8583333333333333, 0.9458333333333333, 0.9]
average performance 0.827197


**`Experiement 2.1: Random, Remove`**

In [ ]:
# hyperparameters
N_CLIENT = 25
N_ADV_RANDOM = 3
N_ADV_OPP = 0
N_ADV_SWAP = 0

# hyperparemeters
TOTAL_TRIAL = 5
TOTAL_ROUND = 30
DETECT_ROUND = 10

# ADV_HANDLE = [None, 'remove', 'reg']
handle = 'remove'

esp = 0.8
min_samples =4
reg_factor = 0.1
metric = 'cosine'
cfl_stats = ExperimentLogger()
model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

for trial in range(TOTAL_TRIAL):
  data = datasets.MNIST(root='./',download=True)
  train_frac = 0.5
  test_frac = 0.2 
  train_num = int(train_frac * len(data))
  test_num = int(test_frac * len(data))
  idcs = np.random.permutation(len(data))
  train_idcs, test_idcs = idcs[:train_num], idcs[train_num:train_num + test_num]
  train_labels = data.train_labels.numpy()
  clients_split = split_image_data(data.train_data[train_idcs], train_labels[train_idcs], n_clients=N_CLIENT, classes_per_client=5,balancedness=1)
  train_trans, val_trans = get_default_data_transforms("EMNIST")
  client_data = [CustomImageDataset(clients_split[i][0].to(torch.float32), clients_split[i][1],transforms=train_trans ) for i in range(len(clients_split))]

  test_data = data.test_data[train_num:train_num+test_num]
  test_labels = train_labels[train_num:train_num+test_num]
  test_data = CustomImageDataset(test_data.to(torch.float32), test_labels, transforms=val_trans)

  # Assign client modes
  clients = [Client(ConvNet, lambda x : torch.optim.SGD(x, lr=0.1, momentum=0.9), client_data[i], idnum=i) 
            for i, dat in enumerate(client_data)]
  client_indx = np.random.permutation(len(clients))
  offset = 0
  adv_random = client_indx[0:N_ADV_RANDOM]
  offset += N_ADV_RANDOM
  adv_opp = client_indx[offset:offset + N_ADV_OPP]
  offset += N_ADV_OPP
  adv_swap = client_indx[offset:offset+N_ADV_SWAP]
  offset += N_ADV_SWAP
  adv_idx = np.concatenate((adv_random,adv_opp,adv_swap)).tolist()
  for i in adv_random:
    clients[i].client_mode = 'random'

  for i in adv_opp:
    clients[i].client_mode = 'opposite'

  for i in adv_swap:
    clients[i].client_mode = 'swap'

  # print out each client and its mode
  for idx, client in enumerate(clients):
    print('{}: {}'.format(idx, client.client_mode))

  server = Server(ConvNet, test_data)
  weights = np.ones(len(clients))
  for round in range(TOTAL_ROUND):
    if round == 0:
      for client in clients:
        client.synchronize_with_server(server)
    
    
    participating_clients = server.select_clients(clients, frac=1.0)

    for client in participating_clients:
        train_stats = client.compute_weight_update(epochs=1)
        client.reset()
      
    if round + 1 == DETECT_ROUND:
        # generate feature matrix for clustering
        client_dW_dicts = [client.dW for client in clients]
        feature_matrix = generate_feature_matrix(client_dW_dicts)

        # detect adversary using clustering
        clustering_label = server.detect_adversary(feature_matrix, esp, min_samples, metric)
        adv_idx_label = np.argwhere(clustering_label == -1).flatten()

        # update clients by handling adversary detected
        weights = handle_adversary(adv_idx_label, handle, weights,reg_factor)

    # aggregate weight updates; copy new weights to clients
    server.aggregate_wieght_updates_weight(clients, weights)
    server.copy_weights(clients)
              
  # evaluate model performance after all the rounds
  acc_clients = [client.evaluate() for client in clients]

  acc_s = []
  for i, acc in enumerate(acc_clients):
    if i not in adv_idx:
      acc_s.append(acc)

  model_performance[handle][trial] = np.mean(np.array(acc_s)) # evaluation result
  print("handle %s, trial %d"%(handle, trial))
  print(acc_s)
print("average performance %f"%(np.mean(model_performance[handle])))

## Experiment A: Model Performance
Compare model that does handle adversary and model that does NOT handle adversary.

Same TOTAL_ROUND -> Different accuracy

In [ ]:
# hyperparemeters
TOTAL_TRIAL = 30
TOTAL_ROUND = 20
DETECT_ROUND = 5

ESP = 0.5
MIN_SAMPLES = 2
METRIC = 'l2'

ADV_HANDLE = [None, 'remove', 'reg']

model_performance = defaultdict(lambda: [None] * TOTAL_TRIAL)

In [ ]:
for handle in ADV_HANDLE:
    for trial in range(TOTAL_TRIAL):
        for round in range(TOTAL_ROUND):
            if round == 0:
                for client in clients:
                    client.synchronize_with_server(server)

                participating_clients = server.select_clients(clients, frac=1.0)

                for client in participating_clients:
                    train_stats = client.compute_weight_update(epochs=1)
                    client.reset()

                if round + 1 == DETECT_ROUND:
                    # generate feature matrix for clustering
                    client_dW_dicts = [client.dW for client in clients]
                    feature_matrix = generate_feature_matrix(client_dW_dicts)

                    # detect adversary using clustering
                    detect_adv_idx = server.detect_adversary(feature_matrix, esp, min_samples, metric)

                    # update clients by handling adversary detected
                    clients = handle_advesary(clients, detect_adv_idx, ADV_HANDLE)

                # aggregate weight updates; copy new weights to clients
                server.aggregate_weight_updates(clients)
                server.copy_weights(clients)
                
        # evaluate model performance after all the rounds
        model_performance[handle][trial] = # evaluation result

In [ ]:
# plots
# TODO


## Experiment B: Convergence Rate
Compare model that does handle adversary and model that does NOT handle adversary.

Same accuracy -> Different round

In [ ]:
# hyperparemeters
TOTAL_TRIAL = 30
MAX_ROUND = 50
DETECT_ROUND = 5

TARGET_ACC = 0.66

ESP = 0.5
MIN_SAMPLES = 2
METRIC = 'l2'

ADV_HANDLE = [None, 'remove', 'reg']

model_round = defaultdict(lambda: [None] * TOTAL_TRIAL)

In [ ]:
for handle in ADV_HANDLE:
    for trial in range(TOTAL_TRIAL):
        for round in range(MAX_ROUND):
            if round == 0:
                for client in clients:
                    client.synchronize_with_server(server)

                participating_clients = server.select_clients(clients, frac=1.0)

                for client in participating_clients:
                    train_stats = client.compute_weight_update(epochs=1)
                    client.reset()

                if round + 1 == DETECT_ROUND:
                    # generate feature matrix for clustering
                    client_dW_dicts = [client.dW for client in clients]
                    feature_matrix = generate_feature_matrix(client_dW_dicts)

                    # detect adversary using clustering
                    detect_adv_idx = server.detect_adversary(feature_matrix, esp, min_samples, metric)

                    # update clients by handling adversary detected
                    clients = handle_advesary(clients, detect_adv_idx, ADV_HANDLE)

                # aggregate weight updates; copy new weights to clients
                server.aggregate_weight_updates(clients)
                server.copy_weights(clients)
                
            # evaluate model performance after each round
            model_performance = # evaluation result
            if model_performance >= TARGET_ACC:
                model_round[handle][trial] = round + 1
                break

In [ ]:
# plots
# TODO
